In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import math

import tensorflow as tf
import pickle

In [ ]:
!nvidia-smi
device_name = tf.test.gpu_device_name()
print("Using device:", device_name)

Fri Dec  6 21:18:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   59C    P8              14W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from transformers import TFBertModel, BertTokenizer

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Load processed data
train = pd.read_parquet('/content/drive/MyDrive/Code + Data/bn_train_data_final_user_cum_helpful.parquet')
val = pd.read_parquet('/content/drive/MyDrive/Code + Data/bn_val_data_final_user_cum_helpful.parquet')
test = pd.read_parquet('/content/drive/MyDrive/Code + Data/bn_test_data_final_user_cum_helpful.parquet')

In [ ]:
cols = train.columns.tolist()
for c in cols:
  print(c)

review_id
user_id
business_id
stars_reviewer
useful
text
name
postal_code
stars_business
categories
total_reviews_for_business
helpful
num_sentences
num_words
pp_text
extractive_summary
similarity_results
weights
search_similarity
experience_similarity
categories_nature
nature
reviewer_cum_helpful


# Process Reviews w/ BERT

In [ ]:
def get_bert_embeddings(text, batch_size = 16):
  embeddings = []
  for start in range (0, len(text), batch_size):
    end = min(start + batch_size, len(text))
    batch = text[start:end]
    # Tokenize
    batch_input = tokenizer(batch, padding=True, truncation=True, return_tensors="tf", max_length = 128)
    # Get embeddings
    output = model(batch_input['input_ids'])
    cls_embeddings = output.last_hidden_state[:,0,:]
    embeddings.append(cls_embeddings.numpy())
  return np.concatenate(embeddings, axis = 0)

In [ ]:
summaries_train = train['extractive_summary'].tolist()
summaries_val = val['extractive_summary'].tolist()
summaries_test = test['extractive_summary'].tolist()

In [ ]:
summary_embeddings_train = get_bert_embeddings(summaries_train)


In [ ]:
with open('train_embeddings.pkl', 'wb') as f:
    pickle.dump(summary_embeddings_train, f)

print("Tensor data saved successfully!")

Tensor data saved successfully!


In [ ]:
summary_embeddings_val= get_bert_embeddings(summaries_val)

In [ ]:
with open('val_embeddings.pkl', 'wb') as f:
    pickle.dump(summary_embeddings_val, f)

print("Tensor data saved successfully!")

In [ ]:
summary_embeddings_test = get_bert_embeddings(summaries_test)


In [ ]:
with open('test_embeddings.pkl', 'wb') as f:
    pickle.dump(summary_embeddings_test, f)

print("Tensor data saved successfully!")

# Process Business Categories with BERT

In [ ]:
# Import category natures
cat_classes = pd.read_csv("/content/drive/MyDrive/Code + Data/category_nature.csv")


In [ ]:
cats = cat_classes['Category'].tolist()


In [ ]:
def get_bert_embeddings_cats(text):
  # Tokenize
  input = tokenizer(text, padding=True, truncation=True, return_tensors="tf", max_length = 128)
  # Get embeddings
  output = model(input)
  embeddings = output.last_hidden_state[:,0,:]
  return embeddings

In [ ]:
cat_embeddings = get_bert_embeddings_cats(cats)

In [ ]:
cat_classes['cat_embeddings'] = [embed.numpy() for embed in cat_embeddings]

In [ ]:
cat_classes.head()

,Category,Nature,cat_embeddings
0,Supernatural Readings,1,"[-0.047289044, 0.10687305, -0.5081789, -0.0552..."
1,Tours,0,"[-0.49038315, -0.093524426, -0.17515579, -0.10..."
2,Hotels & Travel,0,"[-0.023345027, 0.111619025, 0.42126387, -0.189..."
3,Walking Tours,0,"[-0.6811254, 0.08628298, -0.4200291, -0.164102..."
4,Arts & Entertainment,1,"[-0.032628357, 0.16281706, 0.20891456, -0.1340..."


In [ ]:
with open('cat_bert_embeddings.pkl', 'wb') as f:
    pickle.dump(cat_embeddings, f)

print("Tensor data saved successfully!")

Tensor data saved successfully!


Confirm save and load no issues

In [ ]:
# Load the tensor from the pickle file
with open('cat_bert_embeddings.pkl', 'rb') as f:
    cat_bert_embeddings = pickle.load(f)

In [ ]:
cat_bert_embeddings.shape

TensorShape([418, 768])

In [ ]:
loaded_data = cat_bert_embeddings.numpy()

In [ ]:
cat_classes['test'] = loaded_data.tolist()

In [ ]:
cat_classes.head()

,Category,Nature,cat_embeddings,test
0,Supernatural Readings,1,"[-0.047289044, 0.10687305, -0.5081789, -0.0552...","[-0.04728904366493225, 0.1068730503320694, -0...."
1,Tours,0,"[-0.49038315, -0.093524426, -0.17515579, -0.10...","[-0.4903831481933594, -0.09352442622184753, -0..."
2,Hotels & Travel,0,"[-0.023345027, 0.111619025, 0.42126387, -0.189...","[-0.02334502711892128, 0.1116190254688263, 0.4..."
3,Walking Tours,0,"[-0.6811254, 0.08628298, -0.4200291, -0.164102...","[-0.6811254024505615, 0.08628298342227936, -0...."
4,Arts & Entertainment,1,"[-0.032628357, 0.16281706, 0.20891456, -0.1340...","[-0.03262835741043091, 0.1628170609474182, 0.2..."
